<a href="https://colab.research.google.com/github/pb3lab/AI4PD_2025/blob/main/notebooks/tutorial_alphafold2_conformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/engelberger/tutorials-ai4pd-2025/blob/main/tutorial_alphafold2_i89_conformations_v3.ipynb)

# Tutorial: Structure Prediction with AF2, and its conection to co-evolution and protein conformations

**Duration:** 90 minutes  
**Instructor:** Felipe Engelberger  
**Date:** AI4PD Workshop 2025

---

## Learning Objectives

By the end of this tutorial, you will understand:

1. **Coevolution → Structure**: How AlphaFold2's Evoformer leverages evolutionary information to predict structure
2. **MSA → Conformation**: Why MSA presence/absence determines which conformation is predicted
3. **The i89 Case Study**: How removing coevolution signal at the calcium-binding site enables alternative conformation prediction
4. **Conformational Sampling**: Using MSA subsampling and dropout to explore conformational landscapes
5. **Recycling Dynamics**: How AlphaFold2 "changes its mind" about conformations during iterative refinement

## Scientific Background

AlphaFold2's Evoformer module processes Multiple Sequence Alignments (MSAs) to extract coevolution patterns - residues that mutate together through evolution often interact in 3D space. For the i89 protein (Guo, Kortemme et al.), this coevolution signal strongly biases predictions toward the calcium-bound state. By manipulating the MSA input, we can control which conformation AlphaFold2 predicts.

## Tutorial Overview

1. **Setup and Introduction** - Prepare environment and introduce i89 protein
2. **Coevolution Analysis** - Understand the evolutionary signal in the MSA
3. **Structure Predictions** - Compare predictions with/without MSA
4. **Conformational Sampling** - Explore subsampling and dropout strategies
5. **Recycling Analysis** - Track conformational changes through iterations
6. **Results Summary** - Synthesize findings and implications


## Section 1: Environment Setup

First, we'll set up our environment with the AF2 Utils package that provides a simple wrapper around ColabDesign.


In [ ]:
# @title

import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

# ============================================================================
# AUTO-UPDATE UTILS FROM GITHUB
# ============================================================================
# Set this to True to always fetch the latest versions when running in Colab
FORCE_UPDATE = True  # Change to False to use cached versions

def download_latest_utils(force=False):
    """Download the latest utils files from GitHub."""
    base_url = "https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/"
    utils_files = ["af2_utils.py", "logmd_utils.py"]

    for util_file in utils_files:
        if force or not os.path.exists(util_file):
            # Remove old version if forcing update
            if force and os.path.exists(util_file):
                os.remove(util_file)
                print(f"🗑️ Removed old {util_file}")

            # Download latest version
            url = base_url + util_file
            print(f"📥 Downloading latest {util_file} from GitHub...")
            result = os.system(f"wget -q {url} -O {util_file}")

            if result == 0:
                print(f"✅ Successfully downloaded {util_file}")
            else:
                print(f"❌ Failed to download {util_file}")

    # Reload modules if they were already imported
    if 'af2_utils' in sys.modules and force:
        import importlib
        print("🔄 Reloading af2_utils module...")
        importlib.reload(sys.modules['af2_utils'])
    if 'logmd_utils' in sys.modules and force:
        import importlib
        print("🔄 Reloading logmd_utils module...")
        importlib.reload(sys.modules['logmd_utils'])

# Download utils (force update in Colab if FORCE_UPDATE is True)
if IN_COLAB:
    print("=" * 60)
    print("GOOGLE COLAB DETECTED - FETCHING LATEST UTILS")
    print("=" * 60)
    download_latest_utils(force=FORCE_UPDATE)
    print("=" * 60 + "\n")
else:
    # In local environment, only download if missing
    download_latest_utils(force=False)

# Import packages
import af2_utils as af2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# Setup environment
af2.setup_environment(verbose=False)

# Check and install dependencies if needed
status = af2.check_installation(verbose=False)
missing = [k for k, v in status.items() if not v and k != 'environment_setup']
if missing:
    af2.install_dependencies(
        install_colabdesign='colabdesign' in missing,
        install_hhsuite='hhsuite' in missing,
        download_params='alphafold_params' in missing,
        verbose=False
    )

## Section 2: The i89 Protein System

The i89 protein is a 94-residue designed protein that exhibits two distinct conformational states:
- **State 1**: Calcium-bound conformation (typically predicted with MSA)
- **State 2**: Alternative conformation (accessible without MSA)

This conformational switching makes i89 ideal for understanding how AlphaFold2 uses evolutionary information.


In [ ]:
#@title Define i89 Sequence and Load Reference Structures

# i89 protein sequence (94 residues)
I89_SEQUENCE = "GSHMASMEDLQAEARAFLSEEMIAEFKAAFDMFDADGGGDISYKAVGTVFRMLGINPSKEVLDYLKEKIDVDGSGTIDFEEFLVLMVYIMKQDA"

# Download reference structures if needed
if not os.path.exists("state1.pdb") or not os.path.exists("state2.pdb"):
    os.system("wget -q https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/state1.pdb")
    os.system("wget -q https://raw.githubusercontent.com/engelberger/tutorials-ai4pd-2025/main/state2.pdb")

# Load reference CA coordinates for RMSD calculations
state1_coords = af2.load_ca_coords("state1.pdb")  # CA coords for RMSD
state2_coords = af2.load_ca_coords("state2.pdb")  # CA coords for RMSD
ref_rmsd = af2.calculate_rmsd(state1_coords, state2_coords)

print(f"i89 protein: {len(I89_SEQUENCE)} residues")
print(f"RMSD between State 1 and State 2: {ref_rmsd:.2f} Å")
print(f"Calcium-binding loop: residues 85-95")


In [ ]:
#@title Interactive 3D Overlay: State 1 vs State 2
#@markdown Explore the structural differences between the two reference conformations interactively!

if af2.check_logmd():
    overlay_traj = af2.create_reference_overlay_trajectory(
        state1_path="state1.pdb",
        state2_path="state2.pdb",
        sequence=I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,
        verbose=False
    )

    if overlay_traj:
        print("\n" + "="*60)
        print("INTERACTIVE 3D VIEWER:")
        print("="*60)
        print("The viewer below shows both reference structures overlaid.")
        print("You can:")
        print("  • Rotate, zoom, and pan to explore the structures")
        print("  • Use the animation controls to toggle between State 1 and State 2")
        print("  • Observe the structural differences, especially in the calcium-binding loop")
        print(f"\nView at: {overlay_traj.url}")
        print("="*60 + "\n")

        # Display in notebook
        try:
            from IPython.display import display, HTML
            url = overlay_traj.url
            if "?" not in url:
                url += "?"
            else:
                url += "&"
            url += "preset=polymer-cartoon&fps=10"

            html = f'''
            <div style="margin: 20px 0; text-align: center;">
                <h4>Reference Structures Overlay: State 1 vs State 2</h4>
                <iframe
                    src="{url}"
                    width="800"
                    height="600"
                    frameborder="0"
                    style="border: 1px solid #ccc; border-radius: 5px;">
                </iframe>
                <p style="margin-top: 10px;">
                    <a href="{url}" target="_blank">Open in new window</a>
                </p>
            </div>
            '''
            display(HTML(html))
        except ImportError:
            print(f"IPython not available. View at: {overlay_traj.url}")
else:
    print("LogMD not available - install with: pip install logmd")
    print("You can still visualize the structures using PyMOL or other viewers:")



## Section 3: MSA Generation and Coevolution Analysis

### Understanding AlphaFold2's Inputs

AlphaFold2 takes two primary inputs for structure prediction:
1. **MSA (Multiple Sequence Alignment)**: Evolutionary information from homologous sequences
2. **Deletion Matrix**: Tracks insertions/deletions across sequences

The Evoformer module processes these inputs to extract coevolution patterns, which guide structure prediction.


In [ ]:
#@title Generate MSA for i89 (Run once and reuse throughout)

# Check if MSA already exists
if os.path.exists("i89_msa.npy") and os.path.exists("i89_del_matrix.npy"):
    print("Loading existing MSA...")
    msa_full = np.load("i89_msa.npy")
    deletion_matrix = np.load("i89_del_matrix.npy")
    print(f"✓ Loaded MSA with {len(msa_full)} sequences")
else:
    print("Generating MSA using MMseqs2 (this may take 2-3 minutes)...")
    msa_full, deletion_matrix = af2.get_msa(
        sequences=[I89_SEQUENCE],
        jobname="i89_msa",
        mode="unpaired",
        cov=50,
        id=90,
        max_msa=512,
        verbose=False
    )
    # Save for reuse
    np.save("i89_msa.npy", msa_full)
    np.save("i89_del_matrix.npy", deletion_matrix)
    print(f"✓ Generated MSA with {len(msa_full)} sequences")

print(f"MSA shape: {msa_full.shape} (sequences × positions)")
print(f"Deletion matrix shape: {deletion_matrix.shape}")


### Coevolution Analysis: The Key to Understanding Conformational Selection

Coevolution reveals which residues have evolved together to maintain protein function. The Evoformer learns similar patterns to predict which residues interact in 3D space.


In [ ]:

#@title Compare Coevolution: With vs Without MSA
#@markdown This demonstrates how MSA depth affects coevolution signal and ultimately structure prediction

# First, create MSAData for the full MSA
print("Creating MSA data objects for comparison...")

# Convert MSA to sequences list for MSAData
sequences_full = []
for i in range(len(msa_full)):
    # Convert numeric MSA back to amino acid sequence
    seq_indices = msa_full[i]
    aa_map = {0: 'A', 1: 'R', 2: 'N', 3: 'D', 4: 'C', 5: 'Q', 6: 'E', 7: 'G',
              8: 'H', 9: 'I', 10: 'L', 11: 'K', 12: 'M', 13: 'F', 14: 'P', 15: 'S',
              16: 'T', 17: 'W', 18: 'Y', 19: 'V', 20: '-', 21: 'X'}
    seq = ''.join([aa_map.get(idx, 'X') for idx in seq_indices])
    sequences_full.append(np.array(list(seq)))

# Create MSAData for full MSA
msa_data = af2.MSAData(
    array=msa_full,
    deletion_matrix=deletion_matrix,
    sequences=sequences_full,
    neff=len(msa_full),
    length=len(I89_SEQUENCE),
    condition_name="i89 Full MSA (MMseqs2)"
)

# Create visualizer instance
vis = af2.MSACoevolutionVisualizer()

print("Computing coevolution for full MSA...")
coev_full = vis.compute_coevolution(msa_data)

print("\nCreating single-sequence MSA (no homologs) for comparison...")
print("This simulates what happens when we predict without MSA context\n")

# Create single-sequence MSA
msa_single, deletion_matrix_single = af2.create_single_sequence_msa(I89_SEQUENCE)

# Create sequences array for MSAData (single sequence)
sequences_single = [np.array(list(I89_SEQUENCE))]

# Create MSAData for single sequence
msa_data_single = af2.MSAData(
    array=msa_single,
    deletion_matrix=deletion_matrix_single,
    sequences=sequences_single,
    neff=1,
    length=len(I89_SEQUENCE),
    condition_name="i89 Single Sequence (No MSA)"
)

print("Computing coevolution for single-sequence MSA...")
coev_single = vis.compute_coevolution(msa_data_single)

print("\n" + "="*60)
print("COMPARING COEVOLUTION SIGNALS:")
print("="*60)

# Compare using the comparison function
conditions = {
    "With MSA (MMseqs2)": msa_data,
    "Without MSA (Single Sequence)": msa_data_single
}

fig_main, _ = af2.compare_coevolution_conditions(
    conditions,
    show_difference=False,  # Don't create difference figure
    reference_condition="Without MSA (Single Sequence)"
)

print("\nShowing side-by-side comparison...")
fig_main.show()

## Section 4: Structure Predictions - Testing Our Hypothesis

Now we'll test our hypothesis by comparing AlphaFold2 predictions with and without MSA. All predictions automatically save PDB files for every recycle iteration.


In [ ]:
#@title Helper Functions for Consistent Predictions and Analysis

def run_prediction_with_analysis(sequence, msa, deletion_matrix, job_name, num_seeds=3, num_recycles=3):
    """Run predictions with multiple seeds, all 5 models, and save all PDBs."""

    job_folder = af2.create_job_folder(sequence, job_name)

    # Setup model once
    model = af2.setup_model(sequence, verbose=False)

    # Run predictions with all models (5 models × num_seeds × num_recycles)
    print(f"Running {num_seeds} seeds × 5 models × {num_recycles + 1} recycles...")
    all_predictions = af2.predict_with_all_models(
        model,
        msa=msa,
        deletion_matrix=deletion_matrix,
        num_seeds=num_seeds,
        num_recycles=num_recycles,
        seed_start=0,
        save_pdbs=True,
        job_folder=job_folder,
        sequence=sequence,
        models=None,  # Use all 5 default models
        verbose=True
    )

    # Add job metadata to all predictions
    for pred in all_predictions:
        pred['job_folder'] = job_folder
        pred['job_name'] = job_name

    return all_predictions, job_folder

def analyze_conformational_landscape(predictions, state1_coords, state2_coords):
    """Analyze which conformations were sampled."""

    rmsd_data = []
    for pred in predictions:
        ca_coords = pred['structure'][:, 1, :]
        rmsd1 = af2.calculate_rmsd(ca_coords, state1_coords)
        rmsd2 = af2.calculate_rmsd(ca_coords, state2_coords)
        rmsd_data.append({
            'seed': pred['seed'],
            'model_name': pred.get('model_name', 'model_1'),  # Include model name
            'rmsd_state1': rmsd1,
            'rmsd_state2': rmsd2,
            'plddt': pred['metrics']['plddt'] * 100,
            'closer_to': 'State 1' if rmsd1 < rmsd2 else 'State 2'
        })

    return rmsd_data

def plot_recycling_trajectory(predictions, state1_coords, state2_coords, title):
    """Plot how conformations change during recycling."""

    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    for pred in predictions:
        seed = pred['seed']
        trajectory = pred['trajectory']

        # Extract RMSD values for each recycle
        recycles = []
        rmsd1_vals = []
        rmsd2_vals = []

        for step in trajectory:
            ca_coords = step['structure'][:, 1, :]
            recycles.append(step['recycle'])
            rmsd1_vals.append(af2.calculate_rmsd(ca_coords, state1_coords))
            rmsd2_vals.append(af2.calculate_rmsd(ca_coords, state2_coords))

        # Plot trajectories
        axes[0].plot(recycles, rmsd1_vals, 'o-', label=f'Seed {seed}', alpha=0.7)
        axes[1].plot(recycles, rmsd2_vals, 's-', label=f'Seed {seed}', alpha=0.7)

    axes[0].set_xlabel('Recycle', fontsize=12)
    axes[0].set_ylabel('RMSD to State 1 (Å)', fontsize=12)
    axes[0].set_title(f'{title} - Distance to State 1', fontsize=13)
    axes[0].legend(fontsize=10)
    axes[0].grid(True, alpha=0.3)

    axes[1].set_xlabel('Recycle', fontsize=12)
    axes[1].set_ylabel('RMSD to State 2 (Å)', fontsize=12)
    axes[1].set_title(f'{title} - Distance to State 2', fontsize=13)
    axes[1].legend(fontsize=10)
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout(pad=2.0)
    plt.show()

def display_logmd_embedded(trajectory, title="Interactive 3D Structure"):
    """Display LogMD trajectory embedded in notebook cell."""
    if trajectory is None:
        print("No trajectory to display")
        return

    try:
        from IPython.display import display, HTML

        # Enhanced URL with pLDDT coloring
        url = trajectory.url
        if "?" not in url:
            url += "?"
        else:
            url += "&"
        url += "preset=polymer-cartoon&plddt&fps=10"

        # Create embedded iframe
        html = f'''
        <div style="margin: 20px 0; text-align: center;">
            <h4>{title}</h4>
            <iframe
                src="{url}"
                width="900"
                height="650"
                frameborder="0"
                style="border: 1px solid #ccc; border-radius: 5px;">
            </iframe>
            <p style="margin-top: 10px;">
                <a href="{url}" target="_blank">Open in new window</a>
            </p>
        </div>
        '''
        display(HTML(html))
    except ImportError:
        print(f"IPython not available. View trajectory at: {trajectory.url}")

def display_logmd_comparison(traj1, traj2, title1="With MSA", title2="Without MSA"):
    """Display two LogMD trajectories side-by-side."""
    try:
        from IPython.display import display, HTML

        url1 = f"{traj1.url}?preset=polymer-cartoon&plddt&fps=10"
        url2 = f"{traj2.url}?preset=polymer-cartoon&plddt&fps=10"

        html = f'''
        <div style="margin: 20px 0;">
            <h3 style="text-align: center;">Side-by-side Comparison</h3>
            <div style="display: flex; gap: 20px; justify-content: center;">
                <div style="flex: 1; text-align: center;">
                    <h4>{title1}</h4>
                    <iframe src="{url1}" width="600" height="500" frameborder="0"
                            style="border: 1px solid #ccc; border-radius: 5px;"></iframe>
                    <p><a href="{url1}" target="_blank">Full screen</a></p>
                </div>
                <div style="flex: 1; text-align: center;">
                    <h4>{title2}</h4>
                    <iframe src="{url2}" width="600" height="500" frameborder="0"
                            style="border: 1px solid #ccc; border-radius: 5px;"></iframe>
                    <p><a href="{url2}" target="_blank">Full screen</a></p>
                </div>
            </div>
        </div>
        '''
        display(HTML(html))
    except ImportError:
        print(f"IPython not available. View trajectories:")
        print(f"  {title1}: {traj1.url}")
        print(f"  {title2}: {traj2.url}")


**Note on Default Parameters**:

All predictions in this tutorial use the standard AlphaFold2 workflow:
- **3 seeds** (different random initializations for diversity)
- **3 recycles** (iterative refinement iterations)  
- **5 models** (all AlphaFold2 models for ensemble prediction)

These defaults provide a good balance between computational cost and prediction quality. The workflow follows the pattern from `predict.py`:
- **Seeds**: Provide different random initializations → better conformational sampling
- **Recycles**: Iteratively refine the structure → better convergence  
- **Models**: Ensemble predictions from all 5 models → more robust results

**Students are encouraged to explore these parameters later** by modifying the `num_seeds`, `num_recycles`, and `models` arguments in the prediction functions:
- More seeds (e.g., `num_seeds=5`) → better conformational sampling
- More recycles (e.g., `num_recycles=6`) → better convergence but diminishing returns
- Specific models (e.g., `models=["model_1"]`) → targeted analysis of model-specific behavior


### 4.1 Vanilla AlphaFold2 Prediction (With Full MSA)


In [ ]:
#@title Prediction with Full MSA

print("Running predictions with FULL MSA (expecting State 1)...")
predictions_with_msa, folder_with_msa = run_prediction_with_analysis(
    I89_SEQUENCE, msa_full, deletion_matrix, "i89_with_msa", num_seeds=3, num_recycles=3
)

# Analyze results
rmsd_with_msa = analyze_conformational_landscape(predictions_with_msa, state1_coords, state2_coords)

# Summary
print(f"\n PDBs saved in: {folder_with_msa}/")
print(f"\n Total predictions: {len(predictions_with_msa)} (3 seeds × 5 models)")
print("\n Results with MSA (showing final recycle for each):")
for r in rmsd_with_msa:
    print(f"  {r['model_name']}, Seed {r['seed']}: RMSD to State1={r['rmsd_state1']:.2f}Å, State2={r['rmsd_state2']:.2f}Å → {r['closer_to']}")

mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_with_msa])
mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_with_msa])
print(f"\n  Average: State1={mean_rmsd1:.2f}Å, State2={mean_rmsd2:.2f}Å")
print(f"  ✓ Prediction closer to {'State 1' if mean_rmsd1 < mean_rmsd2 else 'State 2'} (as expected with MSA)")


In [ ]:
#@title      Interactive 3D Visualization: Full MSA Predictions
#@markdown Explore all predictions from the Full MSA run in an interactive trajectory

if af2.check_logmd() and 'predictions_with_msa' in locals():
    import logmd_utils

    print("=" * 60)
    print("Creating interactive trajectory for Full MSA predictions...")
    print("=" * 60)

    # Create LogMD trajectory from all predictions
    with_msa_trajectory = logmd_utils.create_trajectory_from_predictions(
        predictions=predictions_with_msa,
        sequence=I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,  # Align all to first structure
        sort_by_rmsd=True,  # Sort by RMSD to State 1
        reference_coords=state1_coords,  # Use State 1 as reference for sorting
        max_structures=60  # Show top 15 structures
    )

    if with_msa_trajectory:
        print("\n Interactive 3D Viewer Ready!")
        print("=" * 60)
        print("This trajectory shows all Full MSA predictions:")
        print("  • Sorted by RMSD to State 1 (best matches first)")
        print("  • All structures aligned for easy comparison")
        print("  • Color coding by pLDDT confidence")
        print("  • Use animation controls to browse through predictions")
        print(f"\nView at: {with_msa_trajectory.url}")
        print("=" * 60 + "\n")

        # Display in notebook
        try:
            from IPython.display import display, HTML
            url = with_msa_trajectory.url
            if "?" not in url:
                url += "?"
            else:
                url += "&"
            url += "preset=polymer-cartoon&fps=5&plddt"

            html = f'''
            <div style="margin: 20px 0; text-align: center;">
                <h3>🧬 Full MSA Predictions - Interactive Trajectory</h3>
                <p style="color: #666;">Showing {min(15, len(predictions_with_msa))} structures sorted by RMSD to State 1</p>
                <iframe
                    src="{url}"
                    width="100%"
                    height="600"
                    frameborder="0"
                    style="border: 2px solid #2194F3; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                </iframe>
                <div style="margin-top: 15px; display: flex; justify-content: center; gap: 20px;">
                    <a href="{url}" target="_blank" style="text-decoration: none; padding: 8px 16px; background: #2194F3; color: white; border-radius: 4px;">🔗 Open in New Tab</a>
                    <span style="color: #666;">Average RMSD to State 1: {mean_rmsd1:.2f} Å</span>
                </div>
            </div>
            '''
            display(HTML(html))
        except ImportError:
            print(f"View trajectory at: {with_msa_trajectory.url}")
    else:
        print("⚠️ Could not create trajectory visualization")
else:
    if not af2.check_logmd():
        print("ℹ️ LogMD not available - install with: pip install logmd")
    else:
        print("⚠️ Run the Full MSA prediction cell above first")


### 4.2 Single Sequence Prediction (No MSA)


In [ ]:
#@title Prediction without MSA (Single Sequence)

# Create single sequence MSA
msa_single, del_single = af2.create_single_sequence_msa(I89_SEQUENCE)

print("Running predictions with SINGLE SEQUENCE (expecting State 2)...")
predictions_no_msa, folder_no_msa = run_prediction_with_analysis(
    I89_SEQUENCE, msa_single, del_single, "i89_no_msa", num_seeds=3, num_recycles=3
)

# Analyze results
rmsd_no_msa = analyze_conformational_landscape(predictions_no_msa, state1_coords, state2_coords)

# Summary
print(f"\n PDBs saved in: {folder_no_msa}/")
print(f"\n Total predictions: {len(predictions_no_msa)} (3 seeds × 5 models)")
print("\n Results without MSA (showing final recycle for each):")
for r in rmsd_no_msa:
    print(f"  {r['model_name']}, Seed {r['seed']}: RMSD to State1={r['rmsd_state1']:.2f}Å, State2={r['rmsd_state2']:.2f}Å → {r['closer_to']}")

mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_no_msa])
mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_no_msa])
print(f"\n  Average: State1={mean_rmsd1:.2f}Å, State2={mean_rmsd2:.2f}Å")
print(f"  ✓ Prediction closer to {'State 2' if mean_rmsd2 < mean_rmsd1 else 'State 1'} (as hypothesized without MSA)")


In [ ]:
#@title  Interactive 3D Visualization: ALL Single Sequence Recycles
#@markdown Load and visualize ALL recycle iterations from disk (not just final predictions)

if af2.check_logmd() and 'folder_no_msa' in locals():
    import logmd_utils
    import os
    import re
    from pathlib import Path

    print("=" * 60)
    print("Loading ALL recycle iterations from disk...")
    print("=" * 60)

    # Find all PDB files
    pdb_folder = Path(folder_no_msa) / "pdb"
    all_pdb_files = sorted(pdb_folder.glob("**/*.pdb"))

    print(f"Found {len(all_pdb_files)} PDB files")

    # Debug: Show first few filenames to understand the pattern
    if all_pdb_files:
        print("Sample filenames:")
        for i, pdb_file in enumerate(all_pdb_files[:3]):
            print(f"  {pdb_file.name}")

    # Load all structures
    all_recycle_predictions = []
    failed_files = []

    for pdb_file in all_pdb_files:
        # Parse filename to extract metadata
        filename = pdb_file.stem

        # Try multiple filename patterns
        patterns = [
            r'(model_\d+)_r(\d+)_seed(\d+)',  # model_1_r0_seed0
            r'(model_\d+)_seed(\d+)_r(\d+)',  # model_1_seed0_r0
            r'seed(\d+)_(model_\d+)_r(\d+)',  # seed0_model_1_r0
        ]

        match = None
        model_name, recycle, seed = None, None, None

        for pattern in patterns:
            match = re.match(pattern, filename)
            if match:
                groups = match.groups()
                # Extract values based on pattern order
                if pattern == r'(model_\d+)_r(\d+)_seed(\d+)':
                    model_name = groups[0]
                    recycle = int(groups[1])
                    seed = int(groups[2])
                elif pattern == r'(model_\d+)_seed(\d+)_r(\d+)':
                    model_name = groups[0]
                    seed = int(groups[1])
                    recycle = int(groups[2])
                elif pattern == r'seed(\d+)_(model_\d+)_r(\d+)':
                    seed = int(groups[0])
                    model_name = groups[1]
                    recycle = int(groups[2])
                break

        if model_name is not None and recycle is not None and seed is not None:
            try:
                # Load structure
                structure, plddt_values = af2.load_pdb(str(pdb_file))

                # Create prediction dict
                pred = {
                    'structure': structure,
                    'plddt': plddt_values,
                    'model_name': model_name,
                    'recycle': recycle,
                    'seed': seed,
                    'filename': filename
                }
                all_recycle_predictions.append(pred)
            except Exception as e:
                failed_files.append((str(pdb_file), str(e)))
                if len(failed_files) <= 3:  # Only print first few failures
                    print(f"  Warning: Failed to load {pdb_file.name}: {e}")
        else:
            failed_files.append((str(pdb_file), "Could not parse filename"))
            if len(failed_files) <= 3:  # Only print first few failures
                print(f"  Warning: Could not parse filename: {filename}")

    print(f"\nSuccessfully loaded {len(all_recycle_predictions)} structures from disk")
    if failed_files:
        print(f"Failed to load/parse {len(failed_files)} files")

    # Only proceed if we have structures
    if all_recycle_predictions:
        # Sort by seed, model, then recycle
        all_recycle_predictions.sort(key=lambda x: (x['seed'], x['model_name'], x['recycle']))

        # Create LogMD trajectory from ALL predictions
        print("\nCreating interactive trajectory with ALL recycles...")

        all_recycles_trajectory = logmd_utils.create_trajectory_from_predictions(
            predictions=all_recycle_predictions,
            sequence=I89_SEQUENCE,
            project="",  # Public upload
            align_structures=True,  # Align all to first structure
            sort_by_rmsd=True,  # Sort by RMSD to State 2
            reference_coords=state2_coords,  # Use State 2 as reference
            max_structures=None  # Show ALL structures (no limit)
        )

        if all_recycles_trajectory:
            print("\n✅ Interactive 3D Viewer Ready!")
            print("=" * 60)
            print(f"This trajectory shows ALL {len(all_recycle_predictions)} structures:")
            print("  • Including all intermediate recycles")
            print("  • Sorted by RMSD to State 2")
            print("  • All structures aligned for easy comparison")
            print("  • Use animation controls to browse through all predictions")
            print(f"\nView at: {all_recycles_trajectory.url}")
            print("=" * 60 + "\n")

            # Calculate statistics
            state2_rmsds = []
            for pred in all_recycle_predictions:
                ca_coords = pred['structure'][:, 1, :]
                rmsd2 = af2.calculate_rmsd(ca_coords, state2_coords)
                state2_rmsds.append(rmsd2)

            if state2_rmsds:  # Check if we have any RMSDs
                mean_rmsd2 = np.mean(state2_rmsds)
                min_rmsd2 = np.min(state2_rmsds)
            else:
                mean_rmsd2 = 0
                min_rmsd2 = 0

            # Display in notebook
            try:
                from IPython.display import display, HTML
                url = all_recycles_trajectory.url
                if "?" not in url:
                    url += "?"
                else:
                    url += "&"
                url += "preset=polymer-cartoon&fps=2&plddt"  # Slower FPS for more structures

                html = f'''
                <div style="margin: 20px 0; text-align: center;">
                    <h3>ALL Single Sequence Predictions - Complete Trajectory</h3>
                    <p style="color: #666;">Showing ALL {len(all_recycle_predictions)} structures (including all recycles)</p>
                    <iframe
                        src="{url}"
                        width="100%"
                        height="600"
                        frameborder="0"
                        style="border: 2px solid #FF5722; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                    </iframe>
                    <div style="margin-top: 15px; display: flex; justify-content: center; gap: 20px;">
                        <a href="{url}" target="_blank" style="text-decoration: none; padding: 8px 16px; background: #FF5722; color: white; border-radius: 4px;">Open in New Tab</a>
                        <span style="color: #666;">Best RMSD to State 2: {min_rmsd2:.2f} Å | Average: {mean_rmsd2:.2f} Å</span>
                    </div>
                    <div style="margin-top: 10px; color: #888; font-size: 0.9em;">
                        Tip: Use arrow keys or animation controls to navigate through all recycles
                    </div>
                </div>
                '''
                display(HTML(html))
            except ImportError:
                print(f"View trajectory at: {all_recycles_trajectory.url}")
        else:
            print(" Could not create trajectory visualization")
    else:
        print("\n No structures were successfully loaded. Please check:")
        print("  1. The PDB files exist in the expected location")
        print("  2. The filename format matches the expected pattern")
        print("  3. Run the Single Sequence prediction cell above to generate the files")
else:
    if not af2.check_logmd():
        print(" LogMD not available - install with: pip install logmd")
    else:
        print(" Run the Single Sequence prediction cell above first")


### 4.3 Conformational Landscape Comparison


In [ ]:
#@title 🎬 Interactive 3D Visualization: Single Sequence Predictions
#@markdown Explore all predictions from the Single Sequence (No MSA) run

if af2.check_logmd() and 'predictions_no_msa' in locals():
    import logmd_utils

    print("=" * 60)
    print("Creating interactive trajectory for Single Sequence predictions...")
    print("=" * 60)

    # Create LogMD trajectory from all predictions
    no_msa_trajectory = logmd_utils.create_trajectory_from_predictions(
        predictions=predictions_no_msa,
        sequence=I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,  # Align all to first structure
        sort_by_rmsd=True,  # Sort by RMSD to State 2
        reference_coords=state2_coords,  # Use State 2 as reference for sorting
        max_structures=60  # Show top 60 structures
    )

    if no_msa_trajectory:
        print("\n✅ Interactive 3D Viewer Ready!")
        print("=" * 60)
        print("This trajectory shows all Single Sequence predictions:")
        print("  • Sorted by RMSD to State 2 (best matches first)")
        print("  • All structures aligned for easy comparison")
        print("  • Color coding by pLDDT confidence")
        print("  • Use animation controls to browse through predictions")
        print(f"\nView at: {no_msa_trajectory.url}")
        print("=" * 60 + "\n")

        # Calculate mean RMSD for display
        mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_no_msa])
        mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_no_msa])

        # Display in notebook
        try:
            from IPython.display import display, HTML
            url = no_msa_trajectory.url
            if "?" not in url:
                url += "?"
            else:
                url += "&"
            url += "preset=polymer-cartoon&fps=5&plddt"

            html = f'''
            <div style="margin: 20px 0; text-align: center;">
                <h3>🧬 Single Sequence Predictions - Interactive Trajectory</h3>
                <p style="color: #666;">Showing {min(60, len(predictions_no_msa))} structures sorted by RMSD to State 2</p>
                <iframe
                    src="{url}"
                    width="100%"
                    height="600"
                    frameborder="0"
                    style="border: 2px solid #FF5722; border-radius: 8px; box-shadow: 0 4px 6px rgba(0,0,0,0.1);">
                </iframe>
                <div style="margin-top: 15px; display: flex; justify-content: center; gap: 20px;">
                    <a href="{url}" target="_blank" style="text-decoration: none; padding: 8px 16px; background: #FF5722; color: white; border-radius: 4px;">🔗 Open in New Tab</a>
                    <span style="color: #666;">Average RMSD to State 2: {mean_rmsd2:.2f} Å</span>
                </div>
            </div>
            '''
            display(HTML(html))
        except ImportError:
            print(f"View trajectory at: {no_msa_trajectory.url}")
    else:
        print("⚠️ Could not create trajectory visualization")
else:
    if not af2.check_logmd():
        print("ℹ️ LogMD not available - install with: pip install logmd")
    else:
        print("⚠️ Run the Single Sequence prediction cell above first")


In [ ]:
#@title Visualize Conformational Landscape (Interactive - All Predictions)
#@markdown This interactive plot shows ALL predictions including intermediate recycles

# Create interactive landscape with ALL predictions (including recycles)
fig = af2.create_interactive_conformational_landscape(
    predictions_with_msa,  # All predictions including trajectory data
    predictions_no_msa,    # All predictions including trajectory data
    state1_coords,
    state2_coords,
    ref_rmsd=ref_rmsd
)

# Display the interactive plot
fig.show()

# Calculate summary statistics from all data
# Extract final predictions for summary (analyze_conformational_landscape is already defined)
rmsd_with_msa_final = analyze_conformational_landscape(predictions_with_msa, state1_coords, state2_coords)
rmsd_no_msa_final = analyze_conformational_landscape(predictions_no_msa, state1_coords, state2_coords)

# Count total predictions including all recycles
total_with_msa = sum(len(pred['trajectory']) for pred in predictions_with_msa)
total_no_msa = sum(len(pred['trajectory']) for pred in predictions_no_msa)

# Print enhanced summary
print("\nConformational Landscape Summary:")
print("=" * 60)
print(f"Total predictions shown (including all recycles): {total_with_msa + total_no_msa}")
print(f"  - With MSA: {total_with_msa} points ({len(predictions_with_msa)} seeds × {len(set(p.get('model_name', 'model_1') for p in predictions_with_msa))} models × {len(predictions_with_msa[0]['trajectory']) if predictions_with_msa else 0} recycles)")
print(f"  - Without MSA: {total_no_msa} points ({len(predictions_no_msa)} seeds × {len(set(p.get('model_name', 'model_1') for p in predictions_no_msa))} models × {len(predictions_no_msa[0]['trajectory']) if predictions_no_msa else 0} recycles)")

# Calculate means from final predictions
state1_mean_with_msa = np.mean([r['rmsd_state1'] for r in rmsd_with_msa_final])
state2_mean_with_msa = np.mean([r['rmsd_state2'] for r in rmsd_with_msa_final])
state1_mean_no_msa = np.mean([r['rmsd_state1'] for r in rmsd_no_msa_final])
state2_mean_no_msa = np.mean([r['rmsd_state2'] for r in rmsd_no_msa_final])

print("\nFinal predictions (after all recycles):")
print(f"  - WITH MSA: Coevolution signal directs to State 1 (Ca-bound)")
print(f"    Mean RMSD: State1={state1_mean_with_msa:.2f} Å, State2={state2_mean_with_msa:.2f} Å")
print(f"  - WITHOUT MSA: No evolutionary bias allows State 2 (alternative)")
print(f"    Mean RMSD: State1={state1_mean_no_msa:.2f} Å, State2={state2_mean_no_msa:.2f} Å")

print("\nInteractive features:")
print("  - Hover over any point to see detailed information")
print("  - Zoom in/out with mouse wheel or touch gestures")
print("  - Pan by clicking and dragging")
print("  - Click legend items to show/hide conditions")
print("  - Each point shows: Model, Seed, Recycle, RMSD values, pLDDT")

## Section 5: Recycling Dynamics - How AlphaFold2 "Changes Its Mind"

During recycling, AlphaFold2 iteratively refines its prediction. Interestingly, it can switch between conformations during this process.


In [ ]:
#@title Comprehensive Recycling Analysis - RMSD Trajectories and Landscape

# Create comprehensive recycling visualization combining RMSD over time and RMSD space
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# Plot 1: RMSD to State 1 over recycles
ax1 = fig.add_subplot(gs[0, 0])
for pred in predictions_with_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    recycles = [step['recycle'] for step in trajectory]
    rmsd1_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state1_coords) for step in trajectory]
    ax1.plot(recycles, rmsd1_vals, 'o-', label=f'With MSA Seed {seed}', alpha=0.7, linewidth=2)
for pred in predictions_no_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    recycles = [step['recycle'] for step in trajectory]
    rmsd1_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state1_coords) for step in trajectory]
    ax1.plot(recycles, rmsd1_vals, 's-', label=f'No MSA Seed {seed}', alpha=0.7, linewidth=2, linestyle='--')
ax1.set_xlabel('Recycle', fontsize=12)
ax1.set_ylabel('RMSD to State 1 (Å)', fontsize=12)
ax1.set_title('RMSD to State 1 During Recycling', fontsize=13)
ax1.legend(fontsize=9, ncol=2)
ax1.grid(True, alpha=0.3)
ax1.tick_params(labelsize=10)

# Plot 2: RMSD to State 2 over recycles
ax2 = fig.add_subplot(gs[0, 1])
for pred in predictions_with_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    recycles = [step['recycle'] for step in trajectory]
    rmsd2_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state2_coords) for step in trajectory]
    ax2.plot(recycles, rmsd2_vals, 'o-', label=f'With MSA Seed {seed}', alpha=0.7, linewidth=2)
for pred in predictions_no_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    recycles = [step['recycle'] for step in trajectory]
    rmsd2_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state2_coords) for step in trajectory]
    ax2.plot(recycles, rmsd2_vals, 's-', label=f'No MSA Seed {seed}', alpha=0.7, linewidth=2, linestyle='--')
ax2.set_xlabel('Recycle', fontsize=12)
ax2.set_ylabel('RMSD to State 2 (Å)', fontsize=12)
ax2.set_title('RMSD to State 2 During Recycling', fontsize=13)
ax2.legend(fontsize=9, ncol=2)
ax2.grid(True, alpha=0.3)
ax2.tick_params(labelsize=10)

# Plot 3: RMSD landscape - With MSA
ax3 = fig.add_subplot(gs[1, 0])
for pred in predictions_with_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    rmsd1_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state1_coords) for step in trajectory]
    rmsd2_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state2_coords) for step in trajectory]
    ax3.plot(rmsd1_vals, rmsd2_vals, 'o-', label=f'Seed {seed}', alpha=0.7, linewidth=2, markersize=6)
    ax3.scatter([rmsd1_vals[0]], [rmsd2_vals[0]], s=200, marker='s', edgecolors='black', linewidths=2, zorder=5, label=f'Start' if seed == 0 else '')
    ax3.scatter([rmsd1_vals[-1]], [rmsd2_vals[-1]], s=200, marker='*', edgecolors='black', linewidths=2, zorder=5, label=f'End' if seed == 0 else '')
ax3.scatter([0], [ref_rmsd], marker='X', s=400, c='red', label=f'State1 vs State2 ({ref_rmsd:.1f}Å)', zorder=6)
ax3.plot([0, 15], [0, 15], 'k--', alpha=0.3)
ax3.set_xlabel('RMSD to State 1 (Å)', fontsize=12)
ax3.set_ylabel('RMSD to State 2 (Å)', fontsize=12)
ax3.set_title('With MSA - Recycling Trajectory in RMSD Space', fontsize=13)
ax3.legend(loc='upper left', fontsize=9)
ax3.grid(True, alpha=0.3)
ax3.tick_params(labelsize=10)

# Plot 4: RMSD landscape - Without MSA
ax4 = fig.add_subplot(gs[1, 1])
for pred in predictions_no_msa:
    seed = pred['seed']
    trajectory = pred['trajectory']
    rmsd1_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state1_coords) for step in trajectory]
    rmsd2_vals = [af2.calculate_rmsd(step['structure'][:, 1, :], state2_coords) for step in trajectory]
    ax4.plot(rmsd1_vals, rmsd2_vals, 's-', label=f'Seed {seed}', alpha=0.7, linewidth=2, markersize=6)
    ax4.scatter([rmsd1_vals[0]], [rmsd2_vals[0]], s=200, marker='s', edgecolors='black', linewidths=2, zorder=5, label=f'Start' if seed == 0 else '')
    ax4.scatter([rmsd1_vals[-1]], [rmsd2_vals[-1]], s=200, marker='*', edgecolors='black', linewidths=2, zorder=5, label=f'End' if seed == 0 else '')
ax4.scatter([0], [ref_rmsd], marker='X', s=400, c='red', label=f'State1 vs State2 ({ref_rmsd:.1f}Å)', zorder=6)
ax4.plot([0, 15], [0, 15], 'k--', alpha=0.3)
ax4.set_xlabel('RMSD to State 1 (Å)', fontsize=12)
ax4.set_ylabel('RMSD to State 2 (Å)', fontsize=12)
ax4.set_title('Without MSA - Recycling Trajectory in RMSD Space', fontsize=13)
ax4.legend(loc='upper left', fontsize=9)
ax4.grid(True, alpha=0.3)
ax4.tick_params(labelsize=10)

plt.suptitle('Comprehensive Recycling Analysis', fontsize=15, y=0.995)
plt.show()

print("\n Recycling Insights:")
print("  • Top row: RMSD evolution over recycles (time series)")
print("  • Bottom row: RMSD landscape showing conformational trajectories")
print("  • Early recycles: Large conformational changes")
print("  • Later recycles: Fine-tuning and convergence")
print("  • Some trajectories cross the diagonal → conformational switch!")
print("  • This shows AlphaFold2 'changing its mind' during refinement")


In [ ]:
#@title Create LogMD Trajectories for Interactive Viewing

if af2.check_logmd():
    print("Creating LogMD trajectories for side-by-side comparison...")

    # Create trajectory for predictions with MSA
    traj_with_msa = af2.create_trajectory_from_ensemble(
        predictions_with_msa,
        I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,
        verbose=False
    )

    # Create trajectory for predictions without MSA
    traj_no_msa = af2.create_trajectory_from_ensemble(
        predictions_no_msa,
        I89_SEQUENCE,
        project="",  # Public upload
        align_structures=True,
        verbose=False
    )

    if traj_with_msa and traj_no_msa:
        print("\n📺 Side-by-side Comparison of Prediction Ensembles:")
        print("  (Viewers are embedded below - compare conformations side-by-side!)")
        print("  You can use the animation controls to step through all seeds and recycles.")

        # Display side-by-side comparison only (individual viewers removed to reduce redundancy)
        display_logmd_comparison(
            traj_with_msa,
            traj_no_msa,
            title1="With MSA (State 1-like)",
            title2="Without MSA (State 2-like)"
        )
else:
    print("LogMD not available - install with: pip install logmd")


## Section 6: Conformational Sampling Strategies

Now let's explore intermediate strategies between full MSA and single sequence to see if we can sample both conformations.


### 6.1 MSA Subsampling


In [ ]:
#@title MSA Subsampling - Reducing Evolutionary Signal

# Test different MSA depths
subsample_sizes = [1, 8, 32, 128]
subsample_results = {}

for n_seq in subsample_sizes:
    print(f"\nTesting with {n_seq} sequences...")

    # Subsample MSA
    if n_seq == 1:
        msa_sub = msa_single
        del_sub = del_single
    else:
        indices = np.random.choice(len(msa_full), min(n_seq, len(msa_full)), replace=False)
        indices[0] = 0  # Keep query sequence
        msa_sub = msa_full[indices]
        del_sub = deletion_matrix[indices]

    # Run predictions
    predictions_sub, folder_sub = run_prediction_with_analysis(
        I89_SEQUENCE, msa_sub, del_sub, f"i89_msa{n_seq}", num_seeds=3, num_recycles=3
    )

    # Analyze
    rmsd_sub = analyze_conformational_landscape(predictions_sub, state1_coords, state2_coords)
    subsample_results[n_seq] = {
        'predictions': predictions_sub,
        'rmsd_data': rmsd_sub,
        'folder': folder_sub
    }

# Visualize results
fig, ax = plt.subplots(figsize=(12, 7))

for n_seq in subsample_sizes:
    rmsd_data = subsample_results[n_seq]['rmsd_data']
    mean_rmsd1 = np.mean([r['rmsd_state1'] for r in rmsd_data])
    mean_rmsd2 = np.mean([r['rmsd_state2'] for r in rmsd_data])

    ax.scatter([n_seq], [mean_rmsd1], s=100, c='steelblue', marker='o', alpha=0.7)
    ax.scatter([n_seq], [mean_rmsd2], s=100, c='coral', marker='s', alpha=0.7)

ax.set_xscale('log')
ax.set_xlabel('Number of MSA Sequences', fontsize=12)
ax.set_ylabel('Mean RMSD (Å)', fontsize=12)
ax.set_title('Conformational Preference vs MSA Depth', fontsize=13)
ax.axhline(y=5, color='gray', linestyle='--', alpha=0.5)
ax.legend(['RMSD to State 1', 'RMSD to State 2'], fontsize=11)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=10)

plt.tight_layout(pad=2.0)
plt.show()

print("\n MSA Subsampling Results:")
print("  • Fewer sequences → Weaker evolutionary bias")
print("  • Transition occurs around 8-32 sequences")
print("  • Single sequence consistently predicts State 2")


### 6.2 Dropout Sampling


In [ ]:
#@title Dropout Sampling - Introducing Stochasticity

print("Testing dropout with full MSA...")
job_folder_dropout = af2.create_job_folder(I89_SEQUENCE, "i89_dropout")
predictions_dropout = []

for seed in range(6):  # More seeds for dropout
    print(f"  Seed {seed}...", end=" ")

    model = af2.setup_model(I89_SEQUENCE, verbose=False)

    # Enable dropout for sampling
    result = af2.predict_structure(
        model,
        msa=msa_full,
        deletion_matrix=deletion_matrix,
        num_recycles=3,
        use_dropout=True,  # Enable dropout
        seed=seed,
        verbose=False
    )

    # Save PDB manually since predict_structure doesn't have save_pdbs
    pdb_path = f"{job_folder_dropout}/pdb/dropout_seed{seed}_r3.pdb"
    os.makedirs(os.path.dirname(pdb_path), exist_ok=True)
    af2.save_pdb(result['structure'], I89_SEQUENCE, pdb_path, result['plddt'])

    result['seed'] = seed
    predictions_dropout.append(result)
    print(f"pLDDT={result['plddt'].mean()*100:.1f}%")

# Analyze dropout results
rmsd_dropout = analyze_conformational_landscape(predictions_dropout, state1_coords, state2_coords)

# Visualize
fig, ax = plt.subplots(figsize=(10, 7))

for r in rmsd_dropout:
    ax.scatter(r['rmsd_state1'], r['rmsd_state2'], s=100, c='purple',
              marker='D', alpha=0.6, label='Dropout' if r['seed']==0 else '')

# Add reference
ax.scatter([0], [ref_rmsd], marker='*', s=500, c='red',
          label=f'State1 vs State2')

max_val = max(ax.get_xlim()[1], ax.get_ylim()[1])
ax.plot([0, max_val], [0, max_val], 'k--', alpha=0.3)

ax.set_xlabel('RMSD to State 1 (Å)', fontsize=12)
ax.set_ylabel('RMSD to State 2 (Å)', fontsize=12)
ax.set_title('Dropout Sampling Results', fontsize=13)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
ax.tick_params(labelsize=10)

plt.tight_layout(pad=2.0)
plt.show()

closer_to_state1 = sum(1 for r in rmsd_dropout if r['closer_to'] == 'State 1')
closer_to_state2 = sum(1 for r in rmsd_dropout if r['closer_to'] == 'State 2')

print(f"\n🎲 Dropout Sampling Results:")
print(f"  • {closer_to_state1} predictions closer to State 1")
print(f"  • {closer_to_state2} predictions closer to State 2")
print(f"  • Dropout introduces variability but MSA still biases toward State 1")
print(f"\n📁 PDBs saved in: {job_folder_dropout}/")


## Section 7: Results Summary and Key Takeaways

### What We Learned

1. **Coevolution → Structure**: The Evoformer learns coevolution patterns from MSAs to predict which residues interact in 3D space.

2. **MSA Controls Conformation**:
   - Strong coevolution signal at the Ca-binding site → State 1 (Ca-bound)
   - No MSA → No evolutionary bias → State 2 (alternative)

3. **Recycling Dynamics**: AlphaFold2 can change conformational preference during recycling iterations, as visualized in the RMSD landscape plots.

4. **Sampling Strategies**:
   - **MSA Subsampling**: Reducing MSA depth weakens evolutionary bias
   - **Dropout**: Adds stochasticity but doesn't overcome strong MSA bias
   - **Single Sequence**: Most reliable for accessing alternative conformations

### Practical Implications

- To sample alternative conformations, consider running predictions without MSA
- MSA depth can be tuned to explore conformational landscapes
- All predictions save PDB files for detailed structural analysis
- LogMD trajectories enable interactive exploration of conformational ensembles
- Recycling analysis reveals when and how AlphaFold2 switches conformations

### Files Generated

All PDB files from this tutorial are saved in job folders named by condition:
- `i89_with_msa/` - Full MSA predictions (all recycles saved)
- `i89_no_msa/` - Single sequence predictions (all recycles saved)
- `i89_msa{N}/` - Subsampled MSA predictions (all recycles saved)
- `i89_dropout/` - Dropout sampling results

Each folder contains PDBs for all seeds and recycles, enabling comprehensive analysis.
